In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-employee-access-challenge/sampleSubmission.csv
/kaggle/input/amazon-employee-access-challenge/train.csv
/kaggle/input/amazon-employee-access-challenge/test.csv


In [2]:
import seaborn as sns
from matplotlib import pyplot as plt
!pip install --user catboost
import catboost
from catboost import *

In [3]:
os.chdir('/kaggle/input/amazon-employee-access-challenge/')

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


### EDA on features  
We don't see any null values present in the training or test data

In [6]:
print(train.isnull().sum())
test.isnull().sum()

ACTION              0
RESOURCE            0
MGR_ID              0
ROLE_ROLLUP_1       0
ROLE_ROLLUP_2       0
ROLE_DEPTNAME       0
ROLE_TITLE          0
ROLE_FAMILY_DESC    0
ROLE_FAMILY         0
ROLE_CODE           0
dtype: int64


id                  0
RESOURCE            0
MGR_ID              0
ROLE_ROLLUP_1       0
ROLE_ROLLUP_2       0
ROLE_DEPTNAME       0
ROLE_TITLE          0
ROLE_FAMILY_DESC    0
ROLE_FAMILY         0
ROLE_CODE           0
dtype: int64

In [7]:
test['ROLE_CODE'].unique()

array([117880, 118865, 118175, 120774, 118425, 125795, 118055, 124196,
       119082, 118322, 117908, 118570, 118892, 118687, 118539, 118332,
       127783, 120346, 118046, 163732, 117888, 119851, 118639, 120954,
       119929, 128231, 126518, 120580, 118232, 118803, 118836, 118926,
       118749, 120099, 132098, 122271, 118794, 118209, 123672, 118525,
       120315, 122275, 124888, 118319, 118399, 118786, 119589, 124436,
       122850, 131851, 120692, 123047, 117973, 118779, 118454, 118261,
       120904, 120359, 118997, 118276, 128424, 119435, 118843, 119067,
       119096, 117948, 129911, 118475, 118644, 118364, 117898, 124154,
       118468, 128765, 120008, 120593, 124578, 120619, 120529, 137970,
       120814, 128095, 118373, 118943, 118914, 153959, 120303, 119353,
       118296, 121145, 121070, 123068, 118982, 120791, 120419, 118807,
       120767, 122069, 118828, 126870, 119951, 122292, 119348, 129231,
       117900, 120562, 133308, 126112, 118899, 121596, 123738, 120173,
      

The columns contain categorical values. A naive bayes, Decision Tree solution as starting point should be taken. 

In [8]:
# !pip install klib
# import klib

In [9]:
df = train.drop(columns=['ACTION'])
train_x = train.drop(columns=['ACTION'])
train_y = train['ACTION'] 
test_x = test.drop(columns=['id'])
# test_y = test['ACTION']

In [10]:
train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(train_x, train_y)

We have our train and test data  
Let's go ahead with models, DT and naive bayes to be applied first

In [12]:
# Logistic Regression
# Not suitable at all due to categorical variables

from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score
model = lm.LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict_proba(test_x)
# model.score(X_test, y_test)
# accuracy_score(predictions, y_test)

# 0.50 accuracy

In [13]:
test.drop(columns=['id']).shape

(58921, 9)

In [14]:
print(f"{X_train.shape}, {X_test.shape}, {test.drop(columns=['id']).shape}")
test_x = test.drop(columns=['id'])
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

(24576, 9), (8193, 9), (58921, 9)


In [15]:
# Naive Bayes doesn't work here due to an extra category in test_data

from sklearn import naive_bayes
model = naive_bayes.CategoricalNB()
# clf = model.fit(X_train, y_train)
# print(f'{clf.score(X_test,y_test)}')
# predictions = clf.predict(test_x)

In [16]:
# Decision Tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=30)
clf = model.fit(X_train, y_train)
print(f'{clf.score(X_test,y_test)}')
predictions = clf.predict(test_x)

# 0.679 Score on test dataset

0.9295740266080801


In [17]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 300)
clf = model.fit(X_train, y_train)
print(f'{clf.score(X_test, y_test)}')
predictions = clf.predict_proba(test_x)

## Score: 0.67333 on Binary Classification 
## Score: 0.85201 on Probabilitistic outputs

0.946783839863298


# Catboost implementation

Catboost is a boosting algorithm. It is better than lightGBM, XGBoost on various tasks or at least on par with them. It's particularly useful here because of it's ability to work with categorical data.  

Catboost tutorial [link](https://github.com/catboost/tutorials/blob/master/classification/classification_tutorial.ipynb)

In [18]:
os.chdir('/kaggle/working')
os.makedirs('/kaggle/working/amazon')
os.listdir('.')

['__notebook__.ipynb', 'amazon']

In [19]:
from catboost.utils import create_cd
feature_names = dict()
for column, name in enumerate(train):
    if column == 0:
        continue
    feature_names[column - 1] = name
dataset_dir = './amazon'   
create_cd(
    label=0, 
    cat_features=list(range(1, train.columns.shape[0])),
    feature_names=feature_names,
    output_path=os.path.join(dataset_dir, 'train.cd')
)

In [20]:
X = train.drop(columns=['ACTION'])
y = train.ACTION
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [21]:
pool1 = Pool(data=X, label=y, cat_features=cat_features)
pool2 = Pool(
    data=os.path.join('/kaggle/input/amazon-employee-access-challenge/', 'train.csv'), 
    delimiter=',', 
    column_description=os.path.join(dataset_dir, 'train.cd'),
    has_header=True
)
pool3 = Pool(data=X, cat_features=cat_features)

# Fastest way to create a Pool is to create it from numpy matrix.
# This way should be used if you want fast predictions
# or fastest way to load the data in python.

X_prepared = X.values.astype(str).astype(object)
# For FeaturesData class categorial features must have type str

pool4 = Pool(
    data=FeaturesData(
        cat_feature_data=X_prepared,
        cat_feature_names=list(X)
    ),
    label=y.values
)

print('Dataset shape')
print('dataset 1:' + str(pool1.shape) +
      '\ndataset 2:' + str(pool2.shape) + 
      '\ndataset 3:' + str(pool3.shape) +
      '\ndataset 4: ' + str(pool4.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 
print('\ndataset 2:')
print(pool2.get_feature_names())
print('\ndataset 3:')
print(pool3.get_feature_names())
print('\ndataset 4:')
print(pool4.get_feature_names())

Dataset shape
dataset 1:(32769, 9)
dataset 2:(32769, 9)
dataset 3:(32769, 9)
dataset 4: (32769, 9)


Column names
dataset 1:
['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']

dataset 2:
['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']

dataset 3:
['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']

dataset 4:
['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']


In [22]:
# train, test split

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

In [23]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=3000,
    random_seed=1,
    learning_rate=0.03,
    loss_function = 'CrossEntropy'
#     custom_loss=['AUC', 'Accuracy']
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=50,
#     plot=True
)

0:	learn: 0.6567130	test: 0.6569737	best: 0.6569737 (0)	total: 85.4ms	remaining: 4m 16s
50:	learn: 0.1955653	test: 0.1924965	best: 0.1924965 (50)	total: 1.16s	remaining: 1m 7s
100:	learn: 0.1680697	test: 0.1601793	best: 0.1601793 (100)	total: 2.61s	remaining: 1m 14s
150:	learn: 0.1591957	test: 0.1506885	best: 0.1506885 (150)	total: 4.53s	remaining: 1m 25s
200:	learn: 0.1556073	test: 0.1485285	best: 0.1485285 (200)	total: 6.57s	remaining: 1m 31s
250:	learn: 0.1533884	test: 0.1474244	best: 0.1474244 (250)	total: 8.51s	remaining: 1m 33s
300:	learn: 0.1514099	test: 0.1467703	best: 0.1467703 (300)	total: 10.5s	remaining: 1m 34s
350:	learn: 0.1492707	test: 0.1463819	best: 0.1463819 (350)	total: 12.5s	remaining: 1m 34s
400:	learn: 0.1474756	test: 0.1456258	best: 0.1456258 (400)	total: 14.6s	remaining: 1m 34s
450:	learn: 0.1459808	test: 0.1453963	best: 0.1453963 (450)	total: 16.8s	remaining: 1m 34s
500:	learn: 0.1445501	test: 0.1450454	best: 0.1450454 (500)	total: 18.8s	remaining: 1m 33s
550:	

In [24]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,RESOURCE,19.985637
1,ROLE_DEPTNAME,16.758267
2,MGR_ID,15.171623
3,ROLE_ROLLUP_2,13.523222
4,ROLE_CODE,9.570468
5,ROLE_FAMILY_DESC,9.026664
6,ROLE_TITLE,5.562219
7,ROLE_ROLLUP_1,5.368687
8,ROLE_FAMILY,5.033214


In [25]:
# Cross Validation
from catboost import cv

params = {}
params['loss_function'] = 'Logloss'
params['iterations'] = 93
params['custom_loss'] = 'AUC'
params['random_seed'] = 63
params['learning_rate'] = 0.5

cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=cat_features),
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
#     plot=True,
    stratified=False,
    verbose=False
)

In [26]:
cv_data

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-AUC-mean,test-AUC-std
0,0,0.302656,0.003539,0.302803,0.003277,0.527207,0.024999
1,1,0.229539,0.010203,0.230381,0.003876,0.623304,0.052296
2,2,0.192070,0.007484,0.197596,0.002449,0.785451,0.022382
3,3,0.179042,0.008345,0.187064,0.001322,0.812158,0.009366
4,4,0.172556,0.007465,0.181942,0.001980,0.827724,0.008529
...,...,...,...,...,...,...,...
88,88,0.159019,0.009416,0.140851,0.002518,0.855637,0.007536
89,89,0.159240,0.009520,0.140555,0.002512,0.855040,0.007574
90,90,0.159546,0.009378,0.140163,0.002543,0.854600,0.007792
91,91,0.159348,0.009303,0.139709,0.002528,0.855070,0.007626


In [27]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

Best validation Logloss score, not stratified: 0.1583±0.0093 on step 68


In [28]:
test = pd.read_csv('/kaggle/input/amazon-employee-access-challenge/test.csv')
predictions = model.predict_proba(test_x)

In [29]:
np.unique(predictions, return_counts= True)

(array([9.16831740e-04, 9.36338929e-04, 1.03231967e-03, ...,
        9.98967680e-01, 9.99063661e-01, 9.99083168e-01]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [30]:
predictions = predictions[:,1]

In [31]:
os.chdir('/kaggle/working/')
os.curdir
sol = pd.DataFrame(predictions)


sol = sol.rename(columns={0:'Action'})
sol.index = range(1, 58922,1)
sol = sol.rename_axis('Id')
sol.to_csv('submission.csv')

We finally get a accuracy score of 0.90126